In [ ]:
'''

Vorgehen

'''

In [ ]:
# Scrapen der pdf-Dateien diesmal mit suche im sichtbaren Text der Homepage bei soup.find_all           ########################
# für zwei Ordner: "pdf_Faelle_und_Tage_nach_Diagnosen_AU" und 
#                  "pdf_Faelle_und_Tage_nach_Diagnosen_KH"         

import requests
from bs4 import BeautifulSoup
import os
#https://www.bundesgesundheitsministerium.de/themen/krankenversicherung/zahlen-und-fakten-zur-krankenversicherung/geschaeftsergebnisse.html#collapse-control-710
# Basis-URL der Webseite
base_url = "https://www.bundesgesundheitsministerium.de"
page_url = base_url + "/themen/krankenversicherung/zahlen-und-fakten-zur-krankenversicherung/geschaeftsergebnisse.html#collapse-control-710"

# Anfrage an die Webseite senden
response = requests.get(page_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Verzeichnis für PDFs erstellen, Code würde nichts machen, wenn der Ordner bereits besteht:
    # os.makedirs("pdf_Faelle _und_Tage_nach_Diagnosen", exist_ok=True)
    # print("Es wurde ein Ordner erstellt")

    # zur besseren Nachvollziehbarkeit print-Ausgabe, ob Ordner schon existiert:
    ordnername_AU = "pdf_Faelle_und_Tage_nach_Diagnosen_AU"

    # Überprüfen, ob der Ordner bereits existiert
    if os.path.exists(ordnername_AU):
        print(f"⚠ Der Ordner '{ordnername_AU}' existiert bereits und wird überschrieben")
        os.makedirs(ordnername_AU, exist_ok=True)

    else:
        os.makedirs(ordnername_AU)
        print(f"✅ Der Ordner '{ordnername_AU}' wurde neu erstellt.")

    print("Ordner AU wurde überprüft")


    ordnername_KH = "pdf_Faelle_und_Tage_nach_Diagnosen_KH"

    # Überprüfen, ob der Ordner bereits existiert
    if os.path.exists(ordnername_KH):
        print(f"⚠ Der Ordner '{ordnername_KH}' existiert bereits und wird überschrieben")
        os.makedirs(ordnername_KH, exist_ok=True)

    else:
        os.makedirs(ordnername_KH)
        print(f"✅ Der Ordner '{ordnername_KH}' wurde neu erstellt.")

    print("Ordner KH wurde überprüft")
    
    schleife = 0
    gefundene_pdf = 1

    for ul in soup.find_all("ul", class_="c-list c-list--unordered"):
        print(ul)
        #href = ul.get('href')
            
        for a in ul.find_all("a"): #, string=" und Tage nach Diagnosen"):
            href = a.get('href')
            
            schleife +=1

            #print(f"1.: Schleife {schleife} wird durchsucht")   # 1.
            #print("2.:", href, "von Schleife", schleife)        # 2.
            text_schleife = a.get_text()
            #print("3.:", text_schleife)                          # 3.      
                
            print("4.: Schleife", schleife, href)                # 4. 


            # Suche nach pdf mit richtigem Namen im Link-Text
            text_suche = "und Tage nach Diagnosen"
            if href and href.endswith(".pdf") and text_suche in text_schleife:
                print(f"Gefundene PDF: {href}")


                # Sortierung in Ordner für AU
                if gefundene_pdf <= 10:
                    
                    pdf_url = (f"{base_url}{href}") 
                    print(pdf_url)
                    
                    pdf_filename = href.split("/")[-1]  # Originaldateiname behalten                
                    pdf_pfad = f"pdf_Faelle_und_Tage_nach_Diagnosen_AU/{gefundene_pdf}_{pdf_filename}"   #  Eigener Ordner für AU (ohne KH)                        
                    gefundene_pdf +=1
                    try:
                        pdf_response = requests.get(pdf_url)
                        if pdf_response.status_code == 200:
                            with open(pdf_pfad, "wb") as f:
                                f.write(pdf_response.content)
                            print(f"Heruntergeladen: {pdf_pfad}")
                            
                        else:
                            print(f"Konnte nicht herunterladen: {pdf_url}")
                    except Exception as e:
                        print(f"Fehler beim Herunterladen {pdf_url}: {e}")


                # Sortierung in Ordner für KH
                elif gefundene_pdf > 10:
                    
                    pdf_url = (f"{base_url}{href}") 
                    print(pdf_url)
                    
                    pdf_filename = href.split("/")[-1]  # Originaldateiname behalten                
                    pdf_pfad = f"pdf_Faelle_und_Tage_nach_Diagnosen_KH/{gefundene_pdf}_{pdf_filename}"   # Eigener Ordner für Krankenhaus-pdfs                       
                    gefundene_pdf +=1
                    try:
                        pdf_response = requests.get(pdf_url)
                        if pdf_response.status_code == 200:
                            with open(pdf_pfad, "wb") as f:
                                f.write(pdf_response.content)
                            print(f"Heruntergeladen: {pdf_pfad}")
                        else:
                            print(f"Konnte nicht herunterladen: {pdf_url}")
                    except Exception as e:
                        print(f"Fehler beim Herunterladen {pdf_url}: {e}")

                

            #else:
                #   print("Fehler beim Abrufen der Webseite.")






In [ ]:
'''
Dieser Python-Code verarbeitet mehrere PDF-Dateien aus einem Ordner, extrahiert relevante Daten und speichert diese als CSV-Datei.

🔍 Schritte im Code
Ordner mit PDFs durchsuchen

Sammelt alle PDFs im Ordner _______________________________________________________??__pdf_AU_Alter_downloads.

Jahr aus Dateinamen extrahieren

Sucht eine 4-stellige Jahreszahl im Dateinamen und speichert sie als Spalte.

PDF-Text mit pdfplumber extrahieren

Öffnet jede PDF-Datei und liest den gesamten Text pro Seite.

Falls eine Seite keinen Text enthält, wird sie übersprungen.

Daten filtern & verarbeiten

Speichert die erste Zeile jeder Seite als Info.

Prüft jede Zeile:

Muss mit einem Buchstaben + zwei Zahlen beginnen (ICD-Code).

Muss exakt 10 Werte enthalten (Zahlenwerte).

Fügt das Jahr & Info als zusätzliche Spalten hinzu.

Daten in pandas.DataFrame speichern

Erstellt eine Tabelle mit 12 Spalten.

Speichert sie als CSV-Datei im PDF-Ordner.

📂 Ausgabe
Die extrahierten Daten werden als output.csv gespeichert.

Sie enthalten die Spalten: "ICD", "Faelle_F", "Tage_F", "Tage_je_Fall_F", "Faelle_M", "Tage_M", "Tage_je_Fall_M", "Faelle_zus", "Tage_zus", "Tage_je_Fall_zus", "Jahr", "Info".

'''

In [ ]:
# Umwandlung der Krankenkassendaten AU = Arbeitsunfähigkeit und speichern in pdf_csv_daten
import pdfplumber
import pandas as pd
import re
import os

def extract_year(file_name):
    words = re.split(r"[_\.\/]", file_name)  # Trennt an "_" und "." gleichzeitig
    for word in words:
        if word.isdigit() and len(word) == 4:  # Prüfen, ob es genau 4 Ziffern sind
            return word  # Jahreszahl gefunden
    return "unbekannt"  # Keine passende Jahreszahl gefunden

# Ordner mit PDFs und csvs:
pdf_folder = "pdf_Faelle_und_Tage_nach_Diagnosen_AU/"        # AU = Arbeitsunfähigkeit
pdf_folder_dest = "pdf_csv_daten"

# zur besseren Nachvollziehbarkeit print-Ausgabe, ob Ordner schon existiert:
ordnername = pdf_folder_dest

# Überprüfen, ob der Ordner bereits existiert
if os.path.exists(ordnername):
    print(f"⚠ Der Ordner '{ordnername}' existiert bereits und wird überschrieben")
    os.makedirs(ordnername, exist_ok=True)
else:
    os.makedirs(ordnername)
    print(f"✅ Der Ordner '{ordnername}' wurde neu erstellt.")

print("Ordner wurde überprüft")

# Liste aller PDF-Dateien im Ordner erstellen
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# Spaltennamen für den DataFrame
columns = ["ICD", "Faelle_F", "Tage_F", "Tage_je_Fall_F",
           "Faelle_M", "Tage_M", "Tage_je_Fall_M",
           "Faelle_zus", "Tage_zus", "Tage_je_Fall_zus",
           "Jahr", "Info", "aus_Datei"]

data = []  # Leere Liste für die extrahierten Daten

for pdf_path in pdf_files:
    datei_name = os.path.basename(pdf_path)
    jahr = extract_year(pdf_path)
    T98_found_in_pdf = False  # Flag, um festzustellen, ob T98 bereits gefunden wurde

    try:
        with pdfplumber.open(pdf_path) as pdf:
            print(f"📄 Verarbeite Datei: {pdf_path} ({len(pdf.pages)} Seiten)")
            for i, page in enumerate(pdf.pages):
                if T98_found_in_pdf:
                    # Wenn T98 bereits gefunden wurde, überspringe weitere Seiten
                    break

                text = page.extract_text()  # extract_text() funktioniert hier super!!!!
                if not text:
                    continue  # Überspringt Seiten ohne erkannten Text
                lines = text.split("\n")
                info_text = lines[0] if lines else ""  # Erste Zeile als "Info"

                for line in lines:
                    if re.match(r"^T98", line):
                        T98_found_in_pdf = True  # Markieren, dass T98 gefunden wurde
                        print(f"🚨 ICD T98 gefunden")
                        values = line.split()  # Trennung nach Leerzeichen
                        if len(values) == 10:  # Sicherstellen, dass genau 10 Werte vorliegen
                            if jahr == "unbekannt":
                                jahr = extract_year(info_text)
                            values.append(jahr)  # Jahr hinzufügen
                            values.append(info_text)  # Info hinzufügen
                            values.append(datei_name)
                            data.append(values)  # alle Werte in die Liste
                        break  # Nach T98 in der aktuellen Seite aufhören
                    elif re.match(r"^[A-Z]\d{2}", line):  # Prüft, ob ICD-Code vorhanden ist
                        values = line.split()  # Trennung nach Leerzeichen
                        if len(values) == 10:  # Sicherstellen, dass genau 10 Werte vorliegen
                            if jahr == "unbekannt":
                                jahr = extract_year(info_text)
                            values.append(jahr)  # Jahr hinzufügen
                            values.append(info_text)  # Info hinzufügen
                            values.append(datei_name)
                            data.append(values)  # alle Werte in die Liste
            # Ende der Seiten-Schleife
    except Exception as e:
        print(f"❌ Fehler bei {pdf_path}: {e}")

# DataFrame erstellen und speichern
df = pd.DataFrame(data, columns=columns)
df.to_csv(os.path.join(pdf_folder_dest, "Krankenkassen_AU.csv"), index=False, encoding="utf-8", sep=";")


In [ ]:
# Umwandlung der Krankenkassendaten (AU mit Krankenhausaufenthalt) und speichern in Ordner "pdf_csv_daten"
import pdfplumber
import pandas as pd
import re
import os

def extract_year(file_name):
    words = re.split(r"[_\.\/]", file_name)  # Trennt an "_" und "." gleichzeitig
    for word in words:
        if word.isdigit() and len(word) == 4:  # Prüfen, ob es genau 4 Ziffern sind
            return word  # Jahreszahl gefunden
    return "unbekannt"  # Keine passende Jahreszahl gefunden

# Ordner mit PDFs und csvs:
pdf_folder = "pdf_Faelle_und_Tage_nach_Diagnosen_KH/"        # AU = Arbeitsunfähigkeit
pdf_folder_dest = "pdf_csv_daten"

# zur besseren Nachvollziehbarkeit print-Ausgabe, ob Ordner schon existiert:
ordnername = pdf_folder_dest

# Überprüfen, ob der Ordner bereits existiert
if os.path.exists(ordnername):
    print(f"⚠ Der Ordner '{ordnername}' existiert bereits und wird überschrieben")
    os.makedirs(ordnername, exist_ok=True)
else:
    os.makedirs(ordnername)
    print(f"✅ Der Ordner '{ordnername}' wurde neu erstellt.")

print("Ordner wurde überprüft")

# Liste aller PDF-Dateien im Ordner erstellen
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# Spaltennamen für den DataFrame
columns = ["ICD", "Faelle_F", "Tage_F", "Tage_je_Fall_F",
           "Faelle_M", "Tage_M", "Tage_je_Fall_M",
           "Faelle_zus", "Tage_zus", "Tage_je_Fall_zus",
           "Jahr", "Info", "aus_Datei"]

data = []  # Leere Liste für die extrahierten Daten

for pdf_path in pdf_files:
    datei_name = os.path.basename(pdf_path)
    jahr = extract_year(pdf_path)
    T98_found_in_pdf = False  # Flag, um festzustellen, ob T98 bereits gefunden wurde

    try:
        with pdfplumber.open(pdf_path) as pdf:
            print(f"📄 Verarbeite Datei: {pdf_path} ({len(pdf.pages)} Seiten)")
            for i, page in enumerate(pdf.pages):
                if T98_found_in_pdf:
                    # Wenn T98 bereits gefunden wurde, überspringe weitere Seiten
                    break

                text = page.extract_text()  # extract_text() funktioniert hier super!!!!
                if not text:
                    continue  # Überspringt Seiten ohne erkannten Text
                lines = text.split("\n")
                info_text = lines[0] if lines else ""  # Erste Zeile als "Info"

                for line in lines:
                    if re.match(r"^T98", line):
                        T98_found_in_pdf = True  # Markieren, dass T98 gefunden wurde
                        print(f"🚨 ICD T98 gefunden")
                        values = line.split()  # Trennung nach Leerzeichen
                        if len(values) == 10:  # Sicherstellen, dass genau 10 Werte vorliegen
                            if jahr == "unbekannt":
                                jahr = extract_year(info_text)
                            values.append(jahr)  # Jahr hinzufügen
                            values.append(info_text)  # Info hinzufügen
                            values.append(datei_name)
                            data.append(values)  # alle Werte in die Liste
                        break  # Nach T98 in der aktuellen Seite aufhören
                    elif re.match(r"^[A-Z]\d{2}", line):  # Prüft, ob ICD-Code vorhanden ist
                        values = line.split()  # Trennung nach Leerzeichen
                        if len(values) == 10:  # Sicherstellen, dass genau 10 Werte vorliegen
                            if jahr == "unbekannt":
                                jahr = extract_year(info_text)
                            values.append(jahr)  # Jahr hinzufügen
                            values.append(info_text)  # Info hinzufügen
                            values.append(datei_name)
                            data.append(values)  # alle Werte in die Liste
            # Ende der Seiten-Schleife
    except Exception as e:
        print(f"❌ Fehler bei {pdf_path}: {e}")

# DataFrame erstellen und speichern
df = pd.DataFrame(data, columns=columns)
df.to_csv(os.path.join(pdf_folder_dest, "Krankenkassen_KH.csv"), index=False, encoding="utf-8", sep=";")


In [ ]:
# pdf zu 2 csv-Dateien abgeschlossen

In [ ]:
# Erweiterung mit Auswertung der Infos:
import pdfplumber
import pandas as pd
import re
import os

# CSV-Datei laden
pdf_folder_dest = "pdf_csv_daten"
csv_file_path = os.path.join(pdf_folder_dest, "Krankenkassen_KH.csv")
df_KH = pd.read_csv(csv_file_path, sep=";")


# dataframe df_KH untersuchen
df_KH.head()
print(df_KH["Info"].unique())
print(df_KH.columns)
unique_pairs_KH = df_KH[["Info", "Jahr"]].drop_duplicates()
print(unique_pairs_KH)

# in df_KH werden die Rentner nicht ausgenommen

In [ ]:
def extract_freiwillig(string):
    string = string.lower()
    if "freiwillig" in string:
        return 1    
    else:
        return 0
        

df_KH["freiwillig"] = df_KH["Info"].apply(extract_freiwillig)

unique_freiwillig_KH = df_KH[["freiwillig", "Jahr"]].drop_duplicates()

print(unique_freiwillig_KH)


In [ ]:
# CSV-Datei laden

pdf_folder_dest = "pdf_csv_daten"
csv_file_path = os.path.join(pdf_folder_dest, "Krankenkassen_AU.csv")
df_AU = pd.read_csv(csv_file_path, sep=";")

## dataframe df_AU untersuchen
df_AU.head()
print(df_AU["Info"].unique())

unique_pairs_AU = df_AU[["Info", "Jahr"]].drop_duplicates()
unique_pairs_AU
print(unique_pairs_AU)


filtered_pairs_AU = unique_pairs_AU[unique_pairs_AU["Info"] == "Arbeitsunfähigkeitsfälle und -tage der Pflichtmitglieder ohne Rentner (GKV)"]
print("\n filtered 'pflicht ohne Rentner':\n",filtered_pairs_AU)

filtered_pairs_AU = unique_pairs_AU[unique_pairs_AU["Info"] == "Arbeitsunfähigkeitsfälle und -tage der Pflichtmitglieder nach Krankheitsarten (absolut)"]
print("\n filtered 'pflicht absolut':\n",filtered_pairs_AU)

# in den Jahren 2013 und 2015 gab es nur Angaben zu AU der Pflichtversicherten ohne Rentner
# in 2014 gab es Angaben sowohl ohne als auch insgesamt

In [ ]:
df_AU["freiwillig"] = df_AU["Info"].apply(extract_freiwillig)

In [ ]:
# wirklich gut vergleichbar sind von KH aus 2011 bis 2017 (alles pflicht absolut)    -> Label: alle pflicht = 1
#                                von AU aus 2013 (alle ohne Rentner)                 -> Label: alle pflicht = 0

#                                             , 2016-2020 (alles pflicht absolut)    -> Label: alle pflicht = 1
# und                            von AU aus 2014 (hier muss differenziert werden)    -> Label: alle pflicht = 0 oder 1
# für 2012, 2014 und 2015 muss nach Info gelabelt werden. Mit der Info darf der gesamte df_AU bearbeitet werden.

In [ ]:
################################################################
# erstmal Label für KH vergeben:
df_KH["Krankenhaus"] = 0
df_AU["Krankenhaus"] = 1
#################################################################

# Label für Pflicht_alle (wie oben recherchiert)
# ---------------------------------------------------------------



# von KH aus 2011 bis 2017 (alles pflicht absolut)    -> Label: alle pflicht = 1
df_KH["Pflicht_alle"] = 1  # hier gab es nur pflichtmitglieder. Rentner wurden nicht ausgenommen

# von AU aus 2013 (alle ohne Rentner)                 -> Label: alle pflicht = 0
df_AU.loc[df_AU["Jahr"] == 2013, "Pflicht_alle"] = 0


# von AU aus 2016-2020 (alles pflicht absolut)        -> Label: alle pflicht = 0
df_AU.loc[df_AU["Jahr"].between(2016, 2020), "Pflicht_alle"] = 1

df_AU.loc[df_AU["Info"] == "Arbeitsunfähigkeitsfälle und -tage der Pflichtmitglieder ohne Rentner (GKV)", "Pflicht_alle"] = 0


df_AU.loc[df_AU["Info"] == "BMG Gesetzliche Krankenversicherung 17.03.2014", "Pflicht_alle"] = 1
df_AU.loc[df_AU["Info"] == "BMG Arbeitsunfähigkeitsfälle und -tage 01.04.2014", "Pflicht_alle"] = 1


# loc[] wird verwendet, um nur bestimmte Zeilen zu ändern

#################################################################
# beide df vereinen
df_alle = pd.concat([df_AU, df_KH], ignore_index=True)

#################################################################
# Männer und Frauen in Zeilen

# Nur relevante Spalten auswählen (ohne "zus"-Spalten)
#df = df[["ICD", "Faelle_F", "Tage_F", "Tage_je_Fall_F",
 #        "Faelle_M", "Tage_M", "Tage_je_Fall_M",
  #       "Pflicht_alle", "Krankenhaus"]]

# Daten für Frauen und Männer getrennt speichern und eine neue Spalte "Geschlecht" hinzufügen
df_frauen = df_alle[["ICD", "Faelle_F", "Tage_F", "Tage_je_Fall_F", "Jahr", "Krankenhaus", "Pflicht_alle", "freiwillig"]].copy()
df_frauen["male"] = 0
df_frauen.rename(columns={"Faelle_F": "Faelle", "Tage_F": "Tage", "Tage_je_Fall_F": "Tage_je_Fall"}, inplace=True)

df_maenner = df_alle[["ICD", "Faelle_M", "Tage_M", "Tage_je_Fall_M", "Jahr", "Krankenhaus", "Pflicht_alle", "freiwillig"]].copy()
df_maenner["male"] = 1
df_maenner.rename(columns={"Faelle_M": "Faelle", "Tage_M": "Tage", "Tage_je_Fall_M": "Tage_je_Fall"}, inplace=True)

# Die beiden DataFrames zusammenfügen
df_KK_deu = pd.concat([df_frauen, df_maenner], ignore_index=True)   # KK = Krankenkassen


# Speichern der aktualisierten Datei
pdf_folder_dest = "pdf_csv_daten"
csv_file_path_erweitert = os.path.join(pdf_folder_dest, "Krankenkassen_alle.csv")
df_KK_deu.to_csv(csv_file_path_erweitert, index=False, encoding="utf-8", sep=";")          # Semikolon ist hier noch wichtig, wegen deutscher Dezimalschreibweise

print(f"✅ Die CSV-Datei wurde erfolgreich erweitert und als {csv_file_path_erweitert} gespeichert!") 

###  Hiermit fast fertig zur Analyse, nur die Punkte als Trennzeichen müssen noch entfernt werden und Kommata durch Punkte ausgetauscht werden.

In [ ]:
df_KK_deu.head() #### 

In [ ]:
print(df_KK_deu[["Jahr", "Pflicht_alle"]].drop_duplicates())



In [ ]:
df_KK_deu.tail()

In [ ]:
nan_rows_AU = df_KK_deu[df_KK_deu["Pflicht_alle"].isna()]
print(nan_rows_AU)

# es gibt keine NaN -> alles prima :)

In [ ]:
df_KK_deu.info()

In [ ]:
for col in ["Faelle", "Tage", "Tage_je_Fall"]:
    # Schritt 1: Tausendertrennzeichen entfernen
    df_KK_deu[col] = df_KK_deu[col].astype(str).str.replace(".", "", regex=False) #regex=False ist hier wichtig, sonst werden die Werte leer



# das Entfernen der punkte hat noch nicht geklappt

In [ ]:
df_KK_deu.head()

In [ ]:
for col in ["Faelle", "Tage", "Tage_je_Fall"]:

    # Schritt 2: Dezimalzeichen umwandeln (Komma → Punkt)
    df_KK_deu[col] = df_KK_deu[col].str.replace(",", ".", regex=False)

    # Schritt 3: Leere Werte in NaN umwandeln, bevor sie als Float gespeichert werden
    df_KK_deu[col] = pd.to_numeric(df_KK_deu[col], errors="coerce")

In [ ]:
df_KK_deu.head()


In [ ]:
df_KK_deu.info()

In [ ]:
# Tabelle in englische Zahlen umgewandelt und jetzt noch als csv gespeichert:

pdf_folder_dest = "pdf_csv_daten"
output_file_path = os.path.join(pdf_folder_dest, "Krankenkassen_alle_Zahlen_okay.csv")
df_KK_deu.to_csv(output_file_path, index=False, sep=",")   # jetzt als Komma, was der default-Einstellung entspricht

print(f"Die umgestaltete Datei wurde erfolgreich als {output_file_path} gespeichert!")

In [ ]:
# im Folgenden erste Auswertungen
# 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Schritt 1: CSV-Datei laden
df = pd.read_csv(r"pdf_csv_daten\Krankenkassen_alle_Zahlen_okay.csv", sep=",")   # Benennung geändert, hier weiter als df aus erstellter csv !!!!!!!!!



In [ ]:
df.info()

In [ ]:
print(df.columns)

In [ ]:

# 🏷️ Nur numerische Spalten auswählen
num_cols = ['Faelle', 'Tage', 'Tage_je_Fall', 'Jahr', 'Krankenhaus',
       'Pflicht_alle', 'freiwillig', 'male']
df_numeric = df[num_cols]  # DataFrame nur mit Zahlen

# 📊 Pairplot erstellen
sns.pairplot(df_numeric)
plt.show()

In [ ]:
# 📊 Korrelation berechnen
corr_matrix = df_numeric.corr()

# 🔥 Heatmap der Korrelationsmatrix
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korrelationsmatrix")
plt.show()

In [ ]:
# Histogramm mit Seaborn
# Schritt 1: CSV-Datei laden
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_filtered_1 = df.loc[(df["Pflicht_alle"] == 1) & (df["freiwillig"] == 0)] # & ist wichtig !!! and funktioniert nicht

plt.figure(figsize=(8,6))
sns.histplot(data=df_filtered_1, x='Jahr', weights='Faelle', hue='male', multiple='dodge', binwidth=1)

# Diagramm anpassen
plt.xlabel("Jahr")
plt.ylabel("Anzahl der Fälle")
plt.title("Histogramm der Fälle nach Jahr und Geschlecht \n Pflichtversicherte mit Rentnern")
#plt.legend(handles=["weiblich", "männlich"], title="Geschlecht")
plt.show()

In [ ]:
#

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Erstelle ein Figure-Objekt mit zwei Subplots nebeneinander
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 Reihe, 2 Spalten

# Erstes Histogramm (Anzahl der Fälle)
sns.histplot(data=df, x="Jahr", weights="Faelle", hue="male", multiple="dodge", binwidth=1, ax=axes[0])
axes[0].set_title("Histogramm der Fälle \n nach Jahr und Geschlecht")
axes[0].set_xlabel("Jahr")
axes[0].set_ylabel("Anzahl der Fälle")
axes[0].legend(title="male(n/y)")

# Zweites Histogramm (Tage je Fall)
sns.histplot(data=df, x="Jahr", weights="Tage_je_Fall", hue="male", multiple="dodge", binwidth=1, ax=axes[1])
axes[1].set_title("Histogramm der Tage \n je Fall nach Jahr und Geschlecht")
axes[1].set_xlabel("Jahr")
axes[1].set_ylabel("Tage je Fall")
axes[1].legend(title="male(n/y)")

# Layout anpassen und anzeigen
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import plotly.express as px


# Interaktiver Scatter-Plot
fig = px.scatter(df_filtered_1, x='Jahr', y='Faelle', color='Krankenhaus',
                 hover_data={'ICD': True, 'Jahr': True, 'Faelle': True, 'Krankenhaus': True,})

# Diagramm anpassen
plt.xlabel("Jahr")
plt.ylabel("Anzahl der Fälle")
plt.title("Histogramm der Fälle nach Jahr und Einweisung ins Krankenhaus (0/1) \n Pflichtversicherte mit Rentnern")
#plt.legend(handles=["weiblich", "männlich"], title="Geschlecht")
# Diagramm anzeigen
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px


# Interaktiver Scatter-Plot
fig = px.scatter(df_filtered_1, x='Jahr', y='Faelle', color='Krankenhaus',
                 hover_data={'ICD': True, 'Jahr': False, 'Faelle': False, 'Krankenhaus': True,  'male': True})

# Diagramm anzeigen
fig.show()

In [ ]:
'''
# Neue Spalte für die ICD-Gruppen nach dem ersten Buchstaben
df['ICD_Gruppe'] = df['ICD'].str[0]  # Extrahiert den ersten Buchstaben

# Interaktiver Scatter-Plot nach ICD-Gruppen
fig = px.scatter(df, x='Jahr', y='Faelle', color='ICD_Gruppe',
                 hover_data={'ICD': True, 'Jahr': True, 'Faelle': True, 'Krankenhaus': True, 'Mitglieder': True})

# Diagramm anzeigen
fig.show()'''

In [ ]:
import pandas as pd
import plotly.express as px


# Funktion zur Zuordnung der ICD-Gruppen
def kategorisiere_icd(icd_code):
    first_letter = icd_code[0]
    if first_letter in ['A', 'B']:
        return "Infektionen & Parasiten"
    elif first_letter in ['C', 'D']:
        return "Tumore"
    elif first_letter in ['E']:
        return "Stoffwechsel & Ernährung"
    elif first_letter in ['F']:
        return "Psychische Störungen"
    elif first_letter in ['G']:
        return "Nervensystem"
    elif first_letter in ['H']:
        return "Auge & Ohr"
    elif first_letter in ['I']:
        return "Herz-Kreislauf"
    elif first_letter in ['J']:
        return "Atmungssystem"
    elif first_letter in ['K']:
        return "Verdauungssystem"
    elif first_letter in ['L']:
        return "Haut & Unterhaut"
    elif first_letter in ['M']:
        return "Muskel-Skelett"
    elif first_letter in ['N']:
        return "Urogenitalsystem"
    elif first_letter in ['O']:
        return "Schwangerschaft & Geburt"
    elif first_letter in ['P']:
        return "Perinatale Erkrankungen"
    elif first_letter in ['Q']:
        return "Angeborene Fehlbildungen"
    elif first_letter in ['R']:
        return "Klinische Symptome"
    elif first_letter in ['S', 'T']:
        return "Verletzungen & Vergiftungen"
    elif first_letter in ['U']:
        return "Spezielle Codes"
    elif first_letter in ['V', 'W', 'X', 'Y']:
        return "Äußere Ursachen"
    elif first_letter in ['Z']:
        return "Gesundheitsfaktoren"
    else:
        return "Unbekannte Kategorie"

# Neue Spalte mit ICD-Kategorien
df['ICD_Kategorie'] = df['ICD'].apply(kategorisiere_icd)

# Interaktiver Scatter-Plot mit zugeordneten ICD-Kategorien
fig = px.scatter(df, x='Jahr', y='Faelle', color='ICD_Kategorie',
                 hover_data={'ICD': True, 'Jahr': False, 'Faelle': False, 'Krankenhaus': True, 'Pflicht_alle': False})

# Diagramm anzeigen
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go



# Einzigartige Werte aus 'Mitglieder' holen
mitgliedsgruppen = df['freiwillig'].unique()

# Anzahl der Spalten setzen (alle Subplots nebeneinander)
fig = make_subplots(rows=1, cols=len(mitgliedsgruppen), shared_xaxes=True, shared_yaxes=True,
                    subplot_titles=[f'Mitglieder: {gruppe}' for gruppe in mitgliedsgruppen])

# Durch jede Mitgliedergruppe iterieren
for i, gruppe in enumerate(mitgliedsgruppen):
    subset = df[df['freiwillig'] == gruppe]  # Daten filtern
    scatter = go.Scatter(
        x=subset['Jahr'],
        y=subset['Faelle'],
        mode='markers',
        marker=dict(size=10, symbol='circle', color=i),
        text=subset['ICD'],
        hoverinfo='text'
    )
    fig.add_trace(scatter, row=1, col=i+1)

# Layout anpassen
fig.update_layout(title="Subplots nebeneinander mit ICD-Kategorien",
                  height=600, showlegend=False)

# Diagramm anzeigen
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Liste der Mitgliedsgruppen
mitglieder_gruppen = ['Pflichtversicherte ohne Rentner', 'freiwillig Versicherte', 'alle Versicherten']

# Subplots erstellen
fig = px.scatter(df, x='Jahr', y='Faelle', color='ICD_Kategorie',
                 facet_col="Mitglieder",  # Mitgliedergruppen werden nebeneinander dargestellt
                 hover_data={'ICD': True, 'Jahr': True, 'Faelle': False, 'Krankenhaus': True, 'Mitglieder': False})

# Layout anpassen
fig.update_layout(title="Scatter-Subplots nach Mitgliedergruppen")

# Diagramm anzeigen
fig.show()

In [ ]:
df.head()

In [ ]:
import os

#pdf_folder_dest = 
output_file_path = os.path.join(pdf_folder_dest, "Krankenkassen_alle_Zahlen_okay_plus-Kategorie.csv")
df.to_csv(output_file_path, index=False, sep=",")

print(f"Die umgestaltete Datei wurde erfolgreich als {output_file_path} gespeichert!")